In [1]:
import requests
import requests_cache
import json
import sys
import os

from bs4 import BeautifulSoup

requests_cache.install_cache('moz',backend='filesystem')


def fetchUrl(url,cookies=None):
    """
    Fetches content from a URL using a GET request with custom User-Agent and cookies.
    """
    # Ensure headers and cookies are dictionaries, if not provided, initialize as empty dicts
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    headers = {"User-Agent": user_agent}
    if cookies is None:
        cookies = {}
    
    # Perform the GET request
    response = requests.get(url, headers=headers, cookies=cookies)
    
    return response

def moz_cookie_manage():
    """save cookie from https://moz.com/domain-analysis"""
    with open("moz-2.json","r") as file:
        cookies = json.loads(file.read())
        cookies = cookies["cookies"]
    return cookies

def queryMoz(domain,cache_disable=False):


    url = f"https://moz.com/domain-analysis?site={domain}"
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    headers = {"User-Agent": user_agent}
    cookies = moz_cookie_manage()

    cookies_moz = {cookie["name"]:cookie["value"] for cookie in cookies}
    columns = ["Domain Authority","Linking Root Domains","Ranking Keywords","Spam Score"]
    if cache_disable:
         with requests_cache.disabled():
            response = fetchUrl(url,cookies=cookies_moz)
    else:
        response = fetchUrl(url,cookies=cookies_moz)
    output_list = [item.text for item in BeautifulSoup(response.content, features="lxml").find_all("h1")[1:]]
    return {key:value for key,value in zip(columns,output_list)}
        #else:
         #   print("Cookies Expired")
        
def create_session():
    """
    Create a session object with a custom User-Agent to persist cookie state.
    """
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    session = requests.Session()
    session.headers.update({"User-Agent": user_agent})
    return session

def query_moz(domain, session):
    """
    Query Moz for domain analysis, using a session to maintain cookie state.
    """
    url = f"https://moz.com/domain-analysis?site={domain}"
    cookies = moz_cookie_manage()

    cookies_moz = {cookie["name"]:cookie["value"] for cookie in cookies}
    
    # Use the session to make a GET request
    response = session.get(url, cookie)
    
    # Check the response status
    if response.status_code != 200:
        print(f"Failed to fetch {url}: Status code {response.status_code}")
        return None

    # Parse the HTML content using BeautifulSoup if needed
    soup = BeautifulSoup(response.text, 'html.parser')
    output_list = [item.text for item in BeautifulSoup(response).find_all("h1")[1:]]
        #output_list = [item.text for item in BeautifulSoup(response.text).find_all("h1")[1:]]
        #if len(output_list) != 0:
    columns = ["Domain Authority","Linking Root Domains","Ranking Keywords","Spam Score"]
    if output_list:
        return {key:value for key,value in zip(columns,output_list)}
    

In [9]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
session = requests.Session()
session.headers.update({"User-Agent": user_agent})

domain = "apple.com"
url = f"https://moz.com/domain-analysis?site={domain}"
cookies = moz_cookie_manage()

cookies_moz = {cookie["name"]:cookie["value"] for cookie in cookies}

# Use the session to make a GET request
response = session.get(url, cookies= cookies_moz)

# Check the response status
if response.status_code != 200:
    print(f"Failed to fetch {url}: Status code {response.status_code}")


# Parse the HTML content using BeautifulSoup if needed
soup = BeautifulSoup(response.text, 'html.parser')
output_list = [item.text for item in BeautifulSoup(response).find_all("h1")[1:]]
    #output_list = [item.text for item in BeautifulSoup(response.text).find_all("h1")[1:]]
    #if len(output_list) != 0:
columns = ["Domain Authority","Linking Root Domains","Ranking Keywords","Spam Score"]
if output_list:
    return {key:value for key,value in zip(columns,output_list)}

Failed to fetch https://moz.com/domain-analysis?site=apple.com: Status code 403


TypeError: object of type 'OriginalResponse' has no len()

In [10]:
soup

<!DOCTYPE html>
<html lang="en-US"><head><title>Just a moment...</title><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="noindex,nofollow" name="robots"/><meta content="width=device-width,initial-scale=1" name="viewport"/><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131}button,html{font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}@media (prefers-color-scheme:dark){body{background-color:#222;color:#d9d9d9}body a{color:#fff}body a:hover{color:#ee730a;text-decoration:underline}body .lds-ring div{border-color:#999 transparent transparent}body .font-red{color:#b20f03}body .big-button,body .pow-button{background-color:#4693ff;color:#1d1d1d}body #challenge-success-text{background-image:url()}body #challenge-error